Rappel du format des données : <br>
mains_3J = [preflop, flop, turn, river, proba, button, small, bigblind,BB, my_stack, stack_J2, stack_J3, my_bet, bet_J2, bet_J3, pot, decision, decision_value]<br>

## Récupération des mains à partir des fichiers

In [1]:
import numpy as np
import torch
import torch.utils.data
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


BATCH_SIZE = 1024


In [16]:
mains_3J = np.load('mains3J.npy')

In [17]:
#creation d'un mask pour detecter les all in ( > 4)
list_allin_3J = np.where(mains_3J[:,17] > 4)[0]

# On supprimer les all-ins : ils seront gérés par un tableau de psuh or fold
mains_3J = np.delete(mains_3J, list_allin_3J, 0)

##  Séparation des 2 classes de raise

In [18]:
list_raise_3J = np.where(np.logical_and(mains_3J[:,17] > 1, mains_3J[:,17] <= 4 ))[0]


mains_3J_onlyraise = mains_3J[list_raise_3J]
test_raise_inf_egal2 = np.where(np.logical_and(mains_3J_onlyraise[:,17] > 1,mains_3J_onlyraise[:,17] <= 2.8))[0]
test_raise2_4 = np.where(np.logical_and(mains_3J_onlyraise[:,17] > 2.8, mains_3J_onlyraise[:,17] <= 4))[0]

mains_3J_onlyraise[test_raise_inf_egal2, 17] = 0  # raise *2.5 
mains_3J_onlyraise[test_raise2_4, 17] = 1    # raise *3.5 

## Séparation des données

In [19]:
nb_features = mains_3J_onlyraise.shape[1]
mains_3J_onlyraise_X = mains_3J_onlyraise[:,:(nb_features-2)]
mains_3J_onlyraise_Y = mains_3J_onlyraise[:, nb_features-1].squeeze()

nb0 = np.where(mains_3J_onlyraise_Y[:] == 0)[0]
nb1 = np.where(mains_3J_onlyraise_Y[:] == 1)[0]

scaler = StandardScaler()
scaler.fit(mains_3J_onlyraise_X[:,[4,8,9,10,11,12,13,14,15]])

mains_3J_onlyraise_X[:,[4,8,9,10,11,12,13,14,15]] = scaler.transform(mains_3J_onlyraise_X[:,[4,8,9,10,11,12,13,14,15]])

mains_3J_onlyraise_X_train, mains_3J_onlyraise_X_val, mains_3J_onlyraise_Y_train,mains_3J_onlyraise_Y_val = train_test_split(
    mains_3J_onlyraise_X, mains_3J_onlyraise_Y, test_size=0.001, stratify=mains_3J_onlyraise_Y)


In [20]:
np.savez('mean_var_3J_raise.npz', mean=scaler.mean_, var=scaler.var_)

## Datasets et dataloaders

In [21]:
class MainsDataset():
    def __init__(self, X, Y):
        self.hands= torch.from_numpy(X).type(torch.cuda.DoubleTensor)
        self.labels= torch.from_numpy(Y).type(torch.cuda.LongTensor)
        
    def __getitem__(self, index):
        hand =  self.hands[index]
        label =  self.labels[index]
        return (hand, label)

    def __len__(self):
        count = len(self.hands)
        return count 

In [22]:
train_mains_3J_onlyraise = MainsDataset(mains_3J_onlyraise_X_train, mains_3J_onlyraise_Y_train)
train_mains_3J_onlyraise_loader = torch.utils.data.DataLoader(train_mains_3J_onlyraise, batch_size=BATCH_SIZE, shuffle=True)

val_mains_3J_onlyraise = MainsDataset(mains_3J_onlyraise_X_val, mains_3J_onlyraise_Y_val)
val_mains_3J_onlyraise_loader = torch.utils.data.DataLoader(val_mains_3J_onlyraise, batch_size=BATCH_SIZE, shuffle=True)


## Construction du réseau de neurones

In [23]:
class NN_3J_onlyraise(nn.Module):

    def __init__(self):
        super(NN_3J_onlyraise, self).__init__()
        self.largeur = 100
        self.input = nn.Linear(16, self.largeur)
        self.hidden = nn.Linear(self.largeur, self.largeur)
        self.final = nn.Linear(self.largeur, 2)


    def forward(self, x):
        x = F.relu(self.input(x))
        x = F.relu(self.hidden(x))
        x = self.final(x)
        return x

NN_3J_onlyraise = NN_3J_onlyraise()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
NN_3J_onlyraise.to(device).type(torch.cuda.DoubleTensor)

NN_3J_onlyraise(
  (input): Linear(in_features=16, out_features=100, bias=True)
  (hidden): Linear(in_features=100, out_features=100, bias=True)
  (final): Linear(in_features=100, out_features=2, bias=True)
)


NN_3J_onlyraise(
  (input): Linear(in_features=16, out_features=100, bias=True)
  (hidden): Linear(in_features=100, out_features=100, bias=True)
  (final): Linear(in_features=100, out_features=2, bias=True)
)

## Entrainement du réseau 

In [24]:

criterion = nn.CrossEntropyLoss()

lr_tab = [0.1, 0.08, 0.06, 0.04, 0.02, 0.01, 0.005]


for lr_ in lr_tab:
    
    optimizer = optim.SGD(NN_3J_onlyraise.parameters(), lr=lr_)
    for epoch in range(30):  # loop over the dataset multiple times

        running_loss = 0.0
        for i, data in enumerate(train_mains_3J_onlyraise_loader, 0):
            # get the inputs
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = NN_3J_onlyraise(inputs)
            loss = criterion(outputs,labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
        print('lr = %.3f  |  %d loss: %.3f' %(lr_ ,epoch + 1,running_loss*BATCH_SIZE))

print('Finished Training')

lr = 0.100  |  1 loss: 6363.039
lr = 0.100  |  2 loss: 5625.752
lr = 0.100  |  3 loss: 5487.351
lr = 0.100  |  4 loss: 5396.971
lr = 0.100  |  5 loss: 5338.491
lr = 0.100  |  6 loss: 5047.561
lr = 0.100  |  7 loss: 4956.926
lr = 0.100  |  8 loss: 4863.602
lr = 0.100  |  9 loss: 4695.759
lr = 0.100  |  10 loss: 4598.933
lr = 0.100  |  11 loss: 4423.952
lr = 0.100  |  12 loss: 4351.579
lr = 0.100  |  13 loss: 4305.692
lr = 0.100  |  14 loss: 4196.855
lr = 0.100  |  15 loss: 4188.475
lr = 0.100  |  16 loss: 4109.818
lr = 0.100  |  17 loss: 4014.130
lr = 0.100  |  18 loss: 3947.567
lr = 0.100  |  19 loss: 3859.680
lr = 0.100  |  20 loss: 3905.049
lr = 0.100  |  21 loss: 3768.160
lr = 0.100  |  22 loss: 3791.171
lr = 0.100  |  23 loss: 3657.313
lr = 0.100  |  24 loss: 3661.256
lr = 0.100  |  25 loss: 3619.030
lr = 0.100  |  26 loss: 3529.148
lr = 0.100  |  27 loss: 3488.400
lr = 0.100  |  28 loss: 3456.890
lr = 0.100  |  29 loss: 3405.080
lr = 0.100  |  30 loss: 3443.198
lr = 0.080  |  1 lo

## Test du réseau

In [25]:
correct = 0
total = 0


classes = ['2.5', '3.5']
class_correct = [0, 0]
class_total = [1, 1]


with torch.no_grad():
    for data in train_mains_3J_onlyraise_loader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = NN_3J_onlyraise(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        c = (predicted == labels).squeeze()
        for i in range(len(labels)):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

print('Accuracy of the network : %d %%' % (100 * correct / total))

print('Accuracy for each class :')

for i in range(2):
    print('%s : %d %%' % (classes[i], 100 * class_correct[i] / class_total[i]))


Accuracy of the network : 87 %
Accuracy for each class :
2.5 : 92 %
3.5 : 73 %


## Sauvegarde du réseau 

In [42]:
model_ft = NN_3J_onlyraise.state_dict()
filename = 'model_theo_3J_onlyraise.pth'
torch.save(model_ft, filename)